Partition the CS table

In [ ]:
CREATE PARTITION FUNCTION pf_PartitionFunction ( date )
AS RANGE RIGHT FOR VALUES ('20160101', '20170101', '20180101', '20190101', '20200101');
GO
CREATE PARTITION SCHEME ps_PartitionScheme
AS PARTITION pf_PartitionFunction
ALL TO ( [PRIMARY] );  
go



In [ ]:
CREATE TABLE [ABCCompany].[Sales].[SalesOrder_PCS](
	[Id] [int] NOT NULL,
	[SalesPerson] [int] NOT NULL,
	[SalesAmount] [decimal](36, 2) NOT NULL,
	[SalesDate] [date] NOT NULL,
	[SalesTerritory] [int] NOT NULL,
	[OrderDescription] [nvarchar](max) NULL,
	[CreateDate] [datetime] NOT NULL,
	[ModifyDate] [datetime] NULL,
	INDEX pcci1 CLUSTERED COLUMNSTORE
) ON ps_PartitionScheme(SalesDate);
GO
INSERT [ABCCompany].[Sales].[SalesOrder_PCS]
SELECT * from [ABCCompany].[Sales].[SalesOrder_CS]

Check compression

In [7]:
SELECT OBJECT_NAME(object_id) AS [ObjectName], 
SUM(Rows) AS [RowCount], data_compression_desc AS [CompressionType]
FROM sys.partitions WITH (NOLOCK)
WHERE index_id < 2 --ignore the partitions from the non-clustered index if any
AND OBJECT_NAME(object_id) NOT LIKE N'sys%'
AND OBJECT_NAME(object_id) NOT LIKE N'queue_%' 
AND OBJECT_NAME(object_id) NOT LIKE N'filestream_tombstone%' 
AND OBJECT_NAME(object_id) NOT LIKE N'fulltext%'
AND OBJECT_NAME(object_id) NOT LIKE N'ifts_comp_fragment%'
AND OBJECT_NAME(object_id) NOT LIKE N'filetable_updates%'
AND OBJECT_NAME(object_id) NOT LIKE N'xml_index_nodes%'
AND OBJECT_NAME(object_id) NOT LIKE N'sqlagent_job%'  
AND OBJECT_NAME(object_id) NOT LIKE N'plan_persist%'  
GROUP BY object_id, data_compression_desc
ORDER BY SUM(Rows) DESC OPTION (RECOMPILE);
------
------


(11 rows affected)

Total execution time: 00:00:00.020

ObjectName,RowCount,CompressionType
SalesOrder_CS,25000000,COLUMNSTORE
SalesOrder,25000000,NONE
SalesOrder_RS,25000000,NONE
SalesOrder_PCS,23800000,COLUMNSTORE
SalesPerson,1001,NONE
SalesTerritory,10,NONE
SalesPersonLevel,4,NONE
SalesTerritoryStatus,3,NONE
persistent_version_store,0,NONE
persistent_version_store_long_term,0,NONE


Check partitioning

In [ ]:
CREATE VIEW Partition_Info
AS
    SELECT  OBJECT_NAME(i.object_id) AS Object_Name ,
            i.index_id AS [Index_ID] ,
            p.partition_number ,
            fg.name AS Filegroup_Name ,
            p.rows ,
            au.total_pages ,
            f.name AS PartitionScheme ,
            CASE f.boundary_value_on_right
              WHEN 1 THEN 'less than'
              ELSE 'less than or equal to'
            END AS 'comparison' ,
            rv.value
    FROM    sys.partitions p
            JOIN sys.indexes i ON p.object_id = i.object_id
                                  AND p.index_id = i.index_id
            JOIN sys.partition_schemes ps ON ps.data_space_id = i.data_space_id
            JOIN sys.partition_functions f ON f.function_id = ps.function_id
            LEFT JOIN sys.partition_range_values rv ON f.function_id = rv.function_id
                                                       AND p.partition_number = rv.boundary_id
            JOIN sys.destination_data_spaces dds ON dds.partition_scheme_id = ps.data_space_id
                                                    AND dds.destination_id = p.partition_number
            JOIN sys.filegroups fg ON dds.data_space_id = fg.data_space_id
            JOIN ( SELECT   container_id ,
                            SUM(total_pages) AS total_pages
                   FROM     sys.allocation_units
                   GROUP BY container_id
                 ) AS au ON au.container_id = p.partition_id
    WHERE   i.index_id < 2 ;
     
GO

In [ ]:
SELECT  * FROM    Partition_Info ORDER BY 1 ,3